In [1]:
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
import timm
import os
import time
import urllib.request

# Pfad zum Bildordner
image_folder = r"C:\Users\egese\Documents\GitHub\ai-doc-checker\vergleich-fotos"

# Bildtransformation (Standard für alle Modelle)
common_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Laden der Klassenbezeichnungen von ImageNet
url = 'https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt'
class_names = urllib.request.urlopen(url).read().decode('utf-8').splitlines()

# Modelle laden (alle vortrainiert auf ImageNet)
resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
efficientnet = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
vit = timm.create_model('vit_base_patch16_224', pretrained=True)

resnet.eval()
efficientnet.eval()
vit.eval()

models_dict = {
    "ResNet50": resnet,
    "EfficientNet-B0": efficientnet,
    "Vision Transformer": vit
}

# Gerät auswählen (GPU falls vorhanden)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for model in models_dict.values():
    model.to(device)

# Ergebnisse speichern
ergebnisse = []

# Alle Bilder aus dem Ordner durchgehen
for image_name in os.listdir(image_folder):
    if not image_name.lower().endswith(('.jpg', '.png', '.jpeg')):
        continue
    image_path = os.path.join(image_folder, image_name)
    image = Image.open(image_path).convert("RGB")
    input_tensor = common_transform(image).unsqueeze(0).to(device)

    bild_resultat = {"Bild": image_name}

    for model_name, model in models_dict.items():
        start = time.time()

        with torch.no_grad():
            output = model(input_tensor)

            if isinstance(output, tuple):  # für ViT
                output = output[0]

            probs = torch.nn.functional.softmax(output, dim=1)[0]
            top1_prob, top1_idx = torch.max(probs, 0)

        dauer = time.time() - start
        vorhergesagt = class_names[top1_idx.item()]

        # Ergebnisse speichern
        bild_resultat[model_name + " Vorhersage"] = vorhergesagt
        bild_resultat[model_name + " Genauigkeit"] = f"{top1_prob.item()*100:.2f}%"
        bild_resultat[model_name + " Zeit"] = f"{dauer:.3f} Sekunden"

    ergebnisse.append(bild_resultat)

# Ergebnisse anzeigen
import pandas as pd

df = pd.DataFrame(ergebnisse)
print(df.to_string(index=False))
df.to_csv("vergleich_ergebnisse.csv", index=False)

               Bild ResNet50 Vorhersage ResNet50 Genauigkeit  ResNet50 Zeit EfficientNet-B0 Vorhersage EfficientNet-B0 Genauigkeit EfficientNet-B0 Zeit Vision Transformer Vorhersage Vision Transformer Genauigkeit Vision Transformer Zeit
        banana.jpeg              banana               60.42% 0.296 Sekunden                     banana                      18.20%       0.178 Sekunden                        banana                         41.62%          0.544 Sekunden
goldenretriever.jpg    golden retriever               91.63% 0.253 Sekunden           golden retriever                      87.34%       0.146 Sekunden              golden retriever                         47.49%          0.526 Sekunden
         laptop.jpg            notebook               53.82% 0.245 Sekunden           desktop computer                      20.01%       0.138 Sekunden              desktop computer                         24.03%          0.534 Sekunden
    pizza-tonno.jpg               pizza             

In [4]:
from IPython.display import display
df = pd.read_csv("vergleich_ergebnisse.csv")
display(df)

,Bild,ResNet50 Vorhersage,ResNet50 Genauigkeit,ResNet50 Zeit,EfficientNet-B0 Vorhersage,EfficientNet-B0 Genauigkeit,EfficientNet-B0 Zeit,Vision Transformer Vorhersage,Vision Transformer Genauigkeit,Vision Transformer Zeit
0,banana.jpeg,banana,60.42%,0.296 Sekunden,banana,18.20%,0.178 Sekunden,banana,41.62%,0.544 Sekunden
1,goldenretriever.jpg,golden retriever,91.63%,0.253 Sekunden,golden retriever,87.34%,0.146 Sekunden,golden retriever,47.49%,0.526 Sekunden
2,laptop.jpg,notebook,53.82%,0.245 Sekunden,desktop computer,20.01%,0.138 Sekunden,desktop computer,24.03%,0.534 Sekunden
3,pizza-tonno.jpg,pizza,99.68%,0.250 Sekunden,pizza,89.49%,0.147 Sekunden,pizza,90.29%,0.540 Sekunden
4,zebra.jpg,zebra,100.00%,0.250 Sekunden,zebra,92.05%,0.139 Sekunden,zebra,94.27%,0.510 Sekunden
